In [46]:
@tdmclient.notebook.sync_var
def set_speed(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right
    
@tdmclient.notebook.sync_var
def get_prox():
    global prox_horizontal
    obst = prox_horizontal
    return obst

In [63]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 28 17:03:23 2021

@author: henry papapatos
"""
import numpy as np
import sys
#sys.path.append(r'C:\Users\Usuario\OneDrive - epfl.ch\Documents\EPFL\Basics of mobile robotics\PROJET\Github\Mobile_robotics')
import math
import cv2 
import time
import importlib
import filters 
import computer_vision
import path_planning
import obstacle_avoidance

TRESH_DIST = 10 #mm
CAMERA = 0 # Camera
SPEED_TO_MMS = 0.3436 #150 speed
YAW_TO_MMS = 0.06086 # -100 100

def main():
         
    try:              
        frame_init = computer_vision.get_image(cap)
        # we need to wait a bit otherwise the image is yellow 
        time.sleep(1)
        
        print("take the rigth image")
        frame_init = computer_vision.get_image(cap)
        
        #frame_init = cv2.cvtColor(frame_init, cv2.COLOR_BGR2RGB)
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        # Extract vertexes, goals, thymio's start position and orientation from first frame
        start_pos, obst_vertexes, goals_pos, px_to_mm = computer_vision.Init(frame_init)
        
        print("Px_to_mm= ", px_to_mm)
        
        px_to_mm = 0.64
        
        computer_vision.display_obstacle(frame_init, start_pos, goals_pos, obst_vertexes)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_obstacle.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        goal_list = path_planning.get_optimal_path(start_pos, goals_pos, obst_vertexes, 
                                                   px_to_mm, draw = True, image = frame_init)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_path.jpg', frame_init) #POUR TUNING PAR ELIOTT

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        frame = computer_vision.get_image(cap)
        
        # The estimated state vector at time t-1 in the global reference frame.
        # [x_t_minus_1, y_t_minus_1, yaw_t_minus_1]
        x_est_t_minus_1, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
        # x_est_t_minus_1 = np.array([0.0,0.0,0.0])
        
        # The control input vector at time t-1 in the global reference frame.
        # [v, yaw_rate]
        u_t_minus_1 = np.array([0 ,0])
         
        # State covariance matrix P_t_minus_1
        P_t_minus_1 = np.array([[0.1,  0,   0],
                                [  0,0.1,   0],
                                [  0,  0, 0.1]])
    
    
        #update le dt car sinon kalman marche pas
        dt = 0.1 # EN SECONDES POUR KALMAN!!!!! same as PI_CLOCK of pid defined in filters?
        sum_error = 0 
        alt_error_pid = 0
        
        state = 0
        obstThrH = 2000
        
        time.sleep(0.1)
        
        while len(goal_list)!=0:
            
            print("goal is: ", goal_list[0])
            
            previous = time.time()
            
            frame = computer_vision.get_image(cap)
            
#             cv2.imshow('running frame', frame)

                
            ##### APPELER get_vision_position##########
            obs_vector_z_t, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
            
            
            computer_vision.display_obstacle(frame, start_pos, goals_pos, obst_vertexes)
            
            computer_vision.display_pos(frame,obs_vector_z_t, px_to_mm, hidden_cam, is_from_camera = True)
            
            
#             print(f'Timestep measurement={obs_vector_z_t}')
    
            # Run the Extended Kalman Filter and store the 
            # near-optimal state and covariance estimates
            optimal_state_estimate_t, covariance_estimate_t = filters.ekf(
                obs_vector_z_t, # Most recent sensor measurement
                x_est_t_minus_1, # Our most recent estimate of the state
                u_t_minus_1, # Our most recent control input
                P_t_minus_1, # Our most recent state covariance matrix
                dt,hidden_cam) # Time interval
            
            computer_vision.display_pos(frame, optimal_state_estimate_t, px_to_mm, 0, is_from_camera = False)
            computer_vision.display_pos(frame, goal_list[0], px_to_mm, 0, is_from_camera = 2)
            
            frame = cv2.resize(frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('running frame', frame)
            if cv2.waitKey(1) == ord('q'):
                break
            
            mask_frame = cv2.resize(mask_frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('mask frame', mask_frame)
            if cv2.waitKey(1) == ord('q'):
                break
                
            if np.linalg.norm(np.array([optimal_state_estimate_t[0],optimal_state_estimate_t[1]])-np.array(goal_list[0])) < TRESH_DIST:
                goal_list.pop(0)
                
            # Get ready for the next timestep by updating the variable values
            x_est_t_minus_1 = optimal_state_estimate_t
            P_t_minus_1 = covariance_estimate_t
            
            #check if obstacle in coming
            obst=get_prox()
            
            if state == 0:
                for i in range(len(obst)-2):
            # switch from goal tracking to obst avoidance if obstacle detected
                    if (obst[i] > obstThrH):
                        print('obst', obst)
                        state = 1
            if state==0:
                #if no
                ######APPELER PID##########
                print('no obst lets pid')
                sum_error, alt_error_pid, v_l,v_r = filters.pid(optimal_state_estimate_t,goal_list[0],sum_error, alt_error_pid,dt, True)
    #             v_l = 50
    #             v_r = 50
                v = (v_l + v_r)*SPEED_TO_MMS/2
                yaw = (v_r - v_l)*YAW_TO_MMS
                u_t_minus_1 = [v, yaw]; # delta_v returned by PID
    #             u_t_minus_1 = 0;
    
            else:  #if yes, obstacle avoidance
                v, state, hiddencam = obstacle_avoidance.obstacle_avoidance(obst, v_l, v_r)
                v_l = v[0]
                v_r = v[1]
                
            set_speed(int(v_l), int(v_r))

            actual = time.time()
            diff = actual-previous
            print("elapsed time is: ", diff)
            if diff<dt:
                time.sleep(dt-diff)
            
    finally: 
        set_speed(0, 0)
        cv2.destroyAllWindows()



In [48]:
print('Connecting to camera...')
cap=cv2.VideoCapture(CAMERA) # Either 0 or 1, front camera or external cam
print('Camera connected !')

Connecting to camera...
Camera connected !


In [49]:
#get the full quality of the camera
print("setting parameters...")
cap.set(3,1920) 
print("first parameter set")
cap.set(4,1080)
print("second parameter set")

setting parameters...
first parameter set
second parameter set


In [45]:
import tdmclient.notebook
print("connecting to thymio")
await tdmclient.notebook.start()

connecting to thymio


In [44]:
await tdmclient.notebook.stop()

In [74]:
importlib.reload(filters)
importlib.reload(path_planning)
importlib.reload(computer_vision)
importlib.reload(obstacle_avoidance)

<module 'obstacle_avoidance' from 'C:\\Users\\papad\\OneDrive\\Documents\\MA1\\Basic_of_mobile_robotics\\project\\Mobile_robotics\\obstacle_avoidance.py'>

In [75]:
main()

Bonjour
take the rigth image
Px_to_mm=  4.139999675777312


100%|██████████| 2/2 [00:00<00:00, 280.44it/s]


goal is:  [253.44  68.48]
no obst lets pid
pos robot is:  [253.544  68.92   58.958]
y: 298.4404761904762
x: 147.09571428571425
Robot_angle: 58.957869080057236
angle goal:  63.76217662381819
Error_angle: -4.804307543760956
Moving and turning, speed l and r: 109.60861508752191 90.39138491247809
Sum_error, Alt_error_pid: -4.804307543760956 0
elapsed time is:  0.18994903564453125
goal is:  [400.64 367.36]
no obst lets pid
pos robot is:  [254.848  69.532  58.747]
y: 297.82797948923974
x: 145.79238523406488
Robot_angle: 58.74711973665057
angle goal:  63.91738441895591
Error_angle: -5.170264682305344
Moving and turning, speed l and r: 110.34052936461069 89.65947063538931
Sum_error, Alt_error_pid: -9.9745722260663 -4.804307543760956
elapsed time is:  0.044534921646118164
goal is:  [400.64 367.36]
no obst lets pid
pos robot is:  [255.096  70.293  59.456]
y: 297.06690958672453
x: 145.54350511661397
Robot_angle: 59.45639063349706
angle goal:  63.898144759109435
Error_angle: -4.441754125612377
Mov

goal is:  [400.64 367.36]
no obst lets pid
pos robot is:  [281.175 127.253  64.316]
y: 240.1072591387829
x: 119.4653637467589
Robot_angle: 64.31643332819816
angle goal:  63.54735914927555
Error_angle: 0.7690741789226081
Moving and turning, speed l and r: 98.46185164215478 101.53814835784522
Sum_error, Alt_error_pid: -48.221304616039255 0.29513606241594204
elapsed time is:  0.06224679946899414
goal is:  [400.64 367.36]
no obst lets pid
pos robot is:  [282.85  130.547  64.136]
y: 236.81253827059015
x: 117.79015848049374
Robot_angle: 64.1360418933043
angle goal:  63.55432514961339
Error_angle: 0.5817167436909045
Moving and turning, speed l and r: 98.83656651261819 101.16343348738181
Sum_error, Alt_error_pid: -47.63958787234835 0.7690741789226081
elapsed time is:  0.040726423263549805
goal is:  [400.64 367.36]
no obst lets pid
pos robot is:  [284.444 134.138  64.065]
y: 233.22230961206822
x: 116.19642347339612
Robot_angle: 64.06539633824583
angle goal:  63.51651650249661
Error_angle: 0.548

goal is:  [400.64 367.36]
Robot not seen by camera3
no obst lets pid
pos robot is:  [368.936 182.624 148.746]
y: 184.73629437928443
x: 31.70350602739984
Robot_angle: 148.74643377913173
angle goal:  80.26204910274875
Error_angle: 68.48438467638299
Turning only, speed l and r: -136.96876935276597 136.96876935276597
Sum_error, Alt_error_pid: 50 66.12273561875352
elapsed time is:  0.0702815055847168
goal is:  [400.64 367.36]
no obst lets pid
pos robot is:  [312.529  83.148 144.398]
y: 284.21237012990645
x: 88.11051639827895
Robot_angle: 144.39803424264417
angle goal:  72.7757027613081
Error_angle: 71.62233148133608
Turning only, speed l and r: -143.24466296267215 143.24466296267215
Sum_error, Alt_error_pid: 50 68.48438467638299
elapsed time is:  0.07862973213195801
goal is:  [400.64 367.36]
no obst lets pid
pos robot is:  [294.939  45.547 145.698]
y: 321.81330824638496
x: 105.70065962818768
Robot_angle: 145.69801319593597
angle goal:  71.81706062525737
Error_angle: 73.8809525706786
Turning

no obst lets pid
pos robot is:  [188.307  29.712  30.956]
y: 337.6480865228751
x: 212.33342384346724
Robot_angle: 30.955648569875088
angle goal:  57.83584913508673
Error_angle: -26.880200565211645
Moving and turning, speed l and r: 153.7604011304233 46.23959886957671
Sum_error, Alt_error_pid: -5.943595065744994 -20.852162363746586
elapsed time is:  0.05795860290527344
goal is:  [400.64 367.36]
no obst lets pid
pos robot is:  [188.565  30.486  28.764]
y: 336.87378940090696
x: 212.07453284843658
Robot_angle: 28.763548899448796
angle goal:  57.808064621772864
Error_angle: -29.044515722324068
Moving and turning, speed l and r: 158.08903144464813 41.910968555351864
Sum_error, Alt_error_pid: -34.988110788069065 -26.880200565211645
elapsed time is:  0.04846310615539551
goal is:  [400.64 367.36]
no obst lets pid
pos robot is:  [192.976  31.935  30.059]
y: 335.42518574620294
x: 207.66359845991593
Robot_angle: 30.05919685675471
angle goal:  58.238119821974955
Error_angle: -28.178922965220245
Mov

goal is:  [400.64 367.36]
no obst lets pid
pos robot is:  [233.939  79.424 138.984]
y: 287.9355003803089
x: 166.70106130705605
Robot_angle: 138.98408377809994
angle goal:  59.93118452247108
Error_angle: 79.05289925562886
Turning only, speed l and r: -158.10579851125772 158.10579851125772
Sum_error, Alt_error_pid: 50 -9.70993481495725
elapsed time is:  0.0604398250579834
goal is:  [400.64 367.36]
Robot not seen by camera3
no obst lets pid
pos robot is:  [234.939  80.424 141.909]
y: 286.9355003803089
x: 165.70106130705605
Robot_angle: 141.90854755757897
angle goal:  59.99419365634731
Error_angle: 81.91435390123166
Turning only, speed l and r: -163.82870780246333 163.82870780246333
Sum_error, Alt_error_pid: 50 79.05289925562886
elapsed time is:  0.05073046684265137
goal is:  [400.64 367.36]
Robot not seen by camera3
no obst lets pid
pos robot is:  [235.939  81.424 144.903]
y: 285.9355003803089
x: 164.70106130705605
Robot_angle: 144.90267058895054
angle goal:  60.05772448784876
Error_angle

elapsed time is:  0.07455563545227051
goal is:  [400.64 367.36]
elapsed time is:  0.06942224502563477
goal is:  [400.64 367.36]
elapsed time is:  0.0649559497833252
goal is:  [400.64 367.36]
elapsed time is:  0.05272936820983887
goal is:  [400.64 367.36]
elapsed time is:  0.06597185134887695
goal is:  [400.64 367.36]
elapsed time is:  0.043669700622558594
goal is:  [400.64 367.36]
elapsed time is:  0.0632481575012207
goal is:  [400.64 367.36]
elapsed time is:  0.057211875915527344
goal is:  [400.64 367.36]
elapsed time is:  0.04869794845581055
goal is:  [400.64 367.36]
elapsed time is:  0.0635080337524414
goal is:  [400.64 367.36]
elapsed time is:  0.04052138328552246
goal is:  [400.64 367.36]
elapsed time is:  0.07368636131286621
goal is:  [400.64 367.36]
elapsed time is:  0.049124717712402344
goal is:  [400.64 367.36]
elapsed time is:  0.06457710266113281
goal is:  [400.64 367.36]
elapsed time is:  0.08775162696838379
goal is:  [400.64 367.36]
elapsed time is:  0.06613373756408691
go

KeyboardInterrupt: 

In [31]:
cap.release()

In [14]:
set_speed(0, 0)